# Introduction

[Start here](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts)

# Key Concepts and Terminology

MCP is often described as the "USC=C for AI applications." Just as USB-C provides a standardized physical and logical interface for connecting various peripherals to computing devices, MCP offers a consistent protocol for linking AI models to external capabilities. This standardization benefits the entire ecosystem:
* users enjoy simpler and more consistent experiences across AI applications
* AI application developers gain easy integration with a growing ecosystem of tools and data sources
* tool and data providers need only create a single implementation that works with multiple AI applications
* the broader ecosystem benefits from increased interoperability, innovation, and reduced fragmentation

## The Integration Problem

The **MxN Integration Problem** refers to the challenge of connecting M different AI applications to N different external tools or data sources without a standardized approach.

## Without MCP (M×N Problem)

Without a protocol like MCP, developers would need to create M×N custom integrations—one for each possible pairing of an AI application with an external capability.

Each AI application would need to integrate with each tool/data source individually. This is a very complex and expensive process which introduces a lot of friction for developers, and high maintenance costs.

Once we have multiple models and multiple tools, the number of integrations becomes too large to manage, each with its own unique interface.

## With MCP (M+N Solution)

MCP transforms this into an M+N problem by providing a standard interface: each AI application implements the client side of MCP once, and each tool/data source implements the server side once. This dramatically reduces integration complexity and maintenance burden.

## Core MCP Terminology

MCP is a standard like HTTP or USB-C, and is a protocol for connecting AI applications to external tools and data sources. Therefore, using standard terminology is crucial to making the MCP work effectively.

When documenting our applications and communicating with the community, we should use the following terminology.

Just like client server relationships in HTTP, MCP has a client and a server.

* **Host:** The user-facing AI application that end-users interact with directly. Examples include Anthropic’s Claude Desktop, AI-enhanced IDEs like Cursor, inference libraries like Hugging Face Python SDK, or custom applications built in libraries like LangChain or smolagents. Hosts initiate connections to MCP Servers and orchestrate the overall flow between user requests, LLM processing, and external tools.

* **Client:** A component within the host application that manages communication with a specific MCP Server. Each Client maintains a 1:1 connection with a single Server, handling the protocol-level details of MCP communication and acting as an intermediary between the Host’s logic and the external Server.

* **Server:** An external program or service that exposes capabilities (Tools, Resources, Prompts) via the MCP protocol.

[Terminology](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts#capabilities)

**Then there's a bunch of other stuff you're better off just reading instead of typing along to.**

# The Communication Protocol

[Resource](https://huggingface.co/learn/mcp-course/en/unit1/communication-protocol)

MCP defines a standardized communication protocol that enables Clients and Servers to exchange messages in a consistent, predictable way. This standardization is critical for interoperability across the community.

## JSON-RPC: The Foundation

At its core, MCP uses **JSON-RPC 2.0** as the message format for all communication between clients and servers. JSON-RPC is a lightweight remote procedure call protocol encoded in JSON, which makes it:
* Human-readable and easy to debug
* Language-agnostic, supporting implementation in any programming environment
* Well-established, with clear specifications and widespread adoption

The protocol defines three types of messages:

### Requests

Sent from Client to Server to initiate an operation. A request message includes:
* A unique ID (`id`)
* The method name to invoke (e.g. `tools/call`)
* Parameters for the method (if any)

Example request:

In [ ]:
import json
import requests

request = {
    'jsonrpc': '2.0',
    'id': 1,
    'params' : {
        'name': 'weather',
        'arguments': {
            'location': 'San Francisco'
        }
    }
}

json_string = json.dumps(request)
json_string_pretty = json.dumps(request, indent=2)
print(json_string_pretty)

### Responses

Sent from Server to Client in reply ot a Request. A Response message includes:
* The same `id` as the corresponding Request
* Either a `result` (for success) or and `error` (for failure)

Example Success Response:

In [6]:
response = {
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "temperature": 62,
    "conditions": "Partly cloudy"
  }
}

json_string = json.dumps(response)
json_string_pretty = json.dumps(response, indent=2)
print(json_string_pretty)

{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "temperature": 62,
    "conditions": "Partly cloudy"
  }
}


Example Error Response:

In [7]:
{
  "jsonrpc": "2.0",
  "id": 1,
  "error": {
    "code": -32602,
    "message": "Invalid location parameter"
  }
}

{'jsonrpc': '2.0',
 'id': 1,
 'error': {'code': -32602, 'message': 'Invalid location parameter'}}

### Notifications
One-way messages that don’t require a response. Typically sent from Server to Client to provide updates or notifications about events.

Example Notification:

In [8]:
{
  "jsonrpc": "2.0",
  "method": "progress",
  "params": {
    "message": "Processing data...",
    "percent": 50
  }
}

{'jsonrpc': '2.0',
 'method': 'progress',
 'params': {'message': 'Processing data...', 'percent': 50}}

## Transport Mechanisms

JSON-RPC defines the message format, but MCP also specifies how these messages are transported between Clients and Servers. Two primary transport mechanisms are supported.

### stdio (Standard Input/Output)

The stdio transport is used for local communication, where the Client and Server run on the same machine:

The Host application launches the Server as a subprocess and communicates with it by writing to its standard input (stdin) and reading from its standard output (stdout).

Use cases for this transport are local tools like file system access or running local scripts.

The main Advantages of this transport are that it’s simple, no network configuration required, and securely sandboxed by the operating system.

### HTTP + SSE (Server-Sent Events) / Streamable HTTP

The HTTP+SSE transport is used for remote communication, where the Client and Server might be on different machines:

Communication happens over HTTP, with the Server using Server-Sent Events (SSE) to push updates to the Client over a persistent connection.

Use cases for this transport are connecting to remote APIs, cloud services, or shared resources.

The main Advantages of this transport are that it works across networks, enables integration with web services, and is compatible with serverless environments.

Recent updates to the MCP standard have introduced or refined “Streamable HTTP,” which offers more flexibility by allowing servers to dynamically upgrade to SSE for streaming when needed, while maintaining compatibility with serverless environments.

Find notes on the **Interaction Lifecycle** [here](https://huggingface.co/learn/mcp-course/en/unit1/communication-protocol#the-interaction-lifecycle).

# Understanding MCP Capabilities

MCP Servers expose a variety of capabilities to Clients through the communication protocol. These capabilities fall into four main categories.

## Tools

Tools are executable functions or actions that the AI model can invoke through the MCP protocol
* **Control:** Tools are typically **model-controlled**, meaning that the LLM decides when to call them based on the user's request and context.
* **Safety:** Due to their ability to perform actions with side effects, tool execution can be dangerous. Therefore, they typically require explicit user approval.
* **Use Cases:** Sending messages, creating tickets, querying APIs, performing calculations.

**Example:** A weather tool that fetches current weather data for a given location:

In [10]:
def get_weather(location: str) -> dict:
    """Get the current weather for a specified location."""
    return {
        "temperature": 72,
        "conditions": "Sunny",
        "humidity": 45
    }

## Resources

Resources provide read-only access to data sources, allowing the AI model to retrieve context without executing complex logic.
* **Control:** Resources are **application-controlled**, meaning the Host application typically decides when to access them.
* **Nature:** They are designed for data retrieval with minimal computation, similar to GET endpoints in REST APIs.
* **Safety:** Since they are read-only, they typically present lower security risks than Tools.
* **Use Cases:** Accessing file contents, retrieving database records, reading configuration information.

**Example:** A resource that provides access to file contents:

In [11]:
def read_file(file_path: str) -> str:
    """Read the contents of a file at the specified path."""
    with open(file_path, 'r') as f:
        return f.read()

## Prompts

Prompts are predefined templates or workflows that guide the interaction between the user, the AI model, and the Server's capabilities.
* **Control:** Prompts are user-controlled, often presented as options in the Host application's UI.
* **Purpose:** They structure interactions for optimal use of available Tools and Resources.
* **Selection:** Users typically select a prompt before the AI model begins processing, setting context for the interaction.
* **Use Cases:** Common workflows, specialized task templates, guided interactions.

**Example:** A prompt template for generating a code review:

In [12]:
def code_review(code: str, language: str) -> list:
    """Generate a code review for the provided code snippet."""
    return [
        {
            "role": "system",
            "content": f"You are a code reviewer examining {language} code. Provide a detailed review highlighting best practices, potential issues, and suggestions for improvement."
        },
        {
            "role": "user",
            "content": f"Please review this {language} code:\n\n```{language}\n{code}\n```"
        }
    ]